In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.313359  [    0/60000]
loss: 2.296536  [ 6400/60000]
loss: 2.279397  [12800/60000]
loss: 2.261836  [19200/60000]
loss: 2.243887  [25600/60000]
loss: 2.211286  [32000/60000]
loss: 2.223738  [38400/60000]
loss: 2.183044  [44800/60000]
loss: 2.183158  [51200/60000]
loss: 2.138742  [57600/60000]
Test Error: 
 Accuracy: 37.6%, Avg loss: 2.137492 

Epoch 2
-------------------------------
loss: 2.157778  [    0/60000]
loss: 2.141641  [ 6400/60000]
loss: 2.082004  [12800/60000]
loss: 2.092048  [19200/60000]
loss: 2.032227  [25600/60000]
loss: 1.970956  [32000/60000]
loss: 2.004086  [38400/60000]
loss: 1.915196  [44800/60000]
loss: 1.921871  [51200/60000]
loss: 1.841871  [57600/60000]
Test Error: 
 Accuracy: 56.4%, Avg loss: 1.845699 

Epoch 3
-------------------------------
loss: 1.884615  [    0/60000]
loss: 1.847124  [ 6400/60000]
loss: 1.734066  [12800/60000]
loss: 1.775471  [19200/60000]
loss: 1.655296  [25600/60000]
loss: 1.616575  [32000/600